# Bước 2: Lập Chỉ Mục Vector Database (Vector DB Indexing)

Notebook này thực hiện bước cốt lõi thứ hai trong quy trình RAG: chuyển đổi dữ liệu văn bản đã được tiền xử lý thành một **cơ sở dữ liệu vector** có thể tìm kiếm được.

**🎯 Mục tiêu:**
1.  **Tải dữ liệu:** Đọc file văn bản sạch đã được tạo ở bước trước (`01_data_preprocessing.ipynb`).
2.  **Chia nhỏ (Chunking):** Áp dụng một chiến lược chunking thông minh, chia văn bản thành các đoạn nhỏ nhưng vẫn giữ được ngữ cảnh logic (ví dụ: theo từng "Điều lệ").
3.  **Tạo Embedding:** Sử dụng một mô hình embedding mạnh mẽ (`multilingual-e5-large`) để chuyển đổi mỗi chunk văn bản thành một vector số học. Vector này đại diện cho ý nghĩa ngữ nghĩa của chunk đó.
4.  **Lập chỉ mục (Indexing):** Lưu trữ các vector này vào **ChromaDB**, một vector database mã nguồn mở. Việc này cho phép chúng ta tìm kiếm các chunk văn bản liên quan nhất đến câu hỏi của người dùng một cách cực kỳ nhanh chóng.
5.  **Kiểm thử:** Kiểm thử bằng cách đặt một câu truy vấn và xem kết quả trả về như nào
5.  **Cải thiện với Re-ranking:** (Tùy chọn nhưng khuyến khích) Sử dụng một mô hình Re-ranker để sắp xếp lại các kết quả tìm kiếm, đưa những đoạn văn bản liên quan nhất lên đầu.

In [22]:
import os
import re
import uuid
import chromadb
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List, Dict
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np

## 0. Thiết Lập & Cấu Hình

Trong ô này, chúng ta định nghĩa tất cả các hằng số và đường dẫn quan trọng sẽ được sử dụng trong suốt notebook. Việc tập trung chúng ở một nơi giúp dễ dàng thay đổi và cấu hình cho các dự án khác nhau.

-   `PROCESSED_DATA_DIR`: Thư mục chứa file văn bản đã qua xử lý.
-   `CHROMA_DB_PATH`: Nơi lưu trữ cơ sở dữ liệu vector trên đĩa.
-   `EMBEDDING_MODEL_NAME`: Tên của mô hình embedding sẽ được tải từ Hugging Face. `multilingual-e5-large` là một mô hình rất mạnh, hiểu được nhiều ngôn ngữ, bao gồm cả tiếng Việt.
-   `RERANKING_MODEL_NAME`: Tên của mô hình re-ranking.
-   `MODELS_PATH`: Thư mục để lưu các mô hình đã tải về, giúp tiết kiệm thời gian cho các lần chạy sau.
-   `CHUNK_SIZE`, `CHUNK_OVERLAP`: Các tham số quan trọng cho việc chia nhỏ văn bản.


In [23]:
PROCESSED_DATA_DIR = 'C:\\Users\\PC\\Desktop\\Project2\\data\\processed'
PROCESSED_TEXT_FILE = os.path.join(PROCESSED_DATA_DIR, 'Quy_che_25.txt')
CHROMA_DB_PATH = 'C:\\Users\\PC\\Desktop\\Project2\\vectorstore\\chroma_db'
EMBEDDING_MODEL_NAME = 'intfloat/multilingual-e5-large'
RERANKING_MODEL_NAME = 'amberoad/bert-multilingual-passage-reranking-msmarco'
MODELS_PATH = 'C:\\Users\\PC\\models\\models\\embeddings'
CHUNK_SIZE = 800 
CHUNK_OVERLAP = 100 


## 1. Định Nghĩa Hàm Chunking Nâng Cao

Đây là một trong những bước quan trọng nhất để quyết định chất lượng của hệ thống RAG. Chúng ta sẽ định nghĩa một hàm `chunk_document_by_dieu_le` được tùy chỉnh:

-   **Nhận diện cấu trúc:** Hàm này sử dụng biểu thức chính quy (regex) để tìm các tiêu đề "Điều X" trong văn bản.
-   **Giữ ngữ cảnh:** Nó chia văn bản thành các phần lớn dựa trên các "Điều". Mỗi chunk sau đó sẽ được gắn **metadata** (siêu dữ liệu) là "Điều" mà nó thuộc về và nguồn gốc văn bản.
-   **Chia nhỏ hơn:** Bên trong mỗi "Điều", nếu nội dung quá dài, hàm sẽ tiếp tục chia nhỏ thành các chunk có kích thước `CHUNK_SIZE` nhưng vẫn cố gắng không cắt ngang giữa các mục được đánh số (1., a), b), ...).

Kết quả là một danh sách các `documents`, mỗi `document` là một dictionary chứa `text` (nội dung chunk) và `metadata` (bao gồm cả thông tin nguồn gốc văn bản).

In [24]:
def chunk_document_by_dieu_le(text: str, chunk_size: int, chunk_overlap: int, source_file: str = None) -> List[Dict]:
    chunks = []
    dieu_le_pattern = r"(Điều\s+\d+)(?::\s*.*?)?\n"
    parts = re.split(dieu_le_pattern, text)
    
    # Xử lý tên nguồn từ tên file
    if source_file:
        source_name = os.path.splitext(os.path.basename(source_file))[0]
    else:
        source_name = "unknown_source"
    
    current_dieu_le_title = "Giới thiệu/Quy định chung" # Cho phần đầu tài liệu trước Điều lệ đầu tiên
    current_content = ""
    
    def split_content_respecting_numbers(content: str, chunk_size: int, chunk_overlap: int) -> List[str]:
        # Pattern để nhận diện các loại điểm đánh số
        number_patterns = [
            r'(?m)^\s*\d+\.',  # Số và dấu chấm (1., 2., ...)
            r'(?m)^\s*[a-z]\)',  # Chữ cái thường và dấu đóng ngoặc (a), b), ...)
            r'(?m)^\s*[ivx]+\)',  # Số La Mã và dấu đóng ngoặc (i), ii), ...)
        ]
        
        # Tìm tất cả các vị trí bắt đầu của các điểm được đánh số và chia đoạn
        sections = []
        last_pos = 0
        content_lines = content.split('\n')
        current_section = []
        
        for line in content_lines:
            # Kiểm tra xem dòng có bắt đầu bằng số hoặc điểm đánh số không
            is_numbered = any(re.match(pattern, line) for pattern in number_patterns)
            
            if is_numbered and current_section:
                # Lưu section hiện tại và bắt đầu section mới
                sections.append('\n'.join(current_section))
                current_section = [line]
            else:
                current_section.append(line)
        
        # Thêm section cuối cùng
        if current_section:
            sections.append('\n'.join(current_section))
        
        # Gộp các section thành chunks có kích thước phù hợp
        chunks = []
        current_chunk = []
        current_length = 0
        
        for section in sections:
            section_length = len(section)
            
            # Nếu section dài hơn chunk_size, chia nhỏ section này
            if section_length > chunk_size:
                # Xử lý nội dung còn lại trong current_chunk trước
                if current_chunk:
                    chunks.append('\n'.join(current_chunk))
                    current_chunk = []
                    current_length = 0
                
                # Chia section dài thành các phần nhỏ hơn
                words = section.split()
                current_part = []
                current_part_length = 0
                
                for word in words:
                    word_length = len(word) + 1  # +1 cho khoảng trắng
                    if current_part_length + word_length > chunk_size and current_part:
                        chunks.append(' '.join(current_part))
                        # Giữ lại một phần overlap cho chunk tiếp theo
                        overlap_size = min(chunk_overlap, current_part_length)
                        current_part = current_part[-int(overlap_size/10):]  # Giữ lại khoảng 10% từ cuối
                        current_part_length = sum(len(w) + 1 for w in current_part)
                    current_part.append(word)
                    current_part_length += word_length
                
                if current_part:
                    chunks.append(' '.join(current_part))
            
            # Nếu thêm section hiện tại vào chunk hiện tại sẽ vượt quá kích thước
            elif current_length + section_length > chunk_size and current_chunk:
                chunks.append('\n'.join(current_chunk))
                current_chunk = [section]
                current_length = section_length
            else:
                current_chunk.append(section)
                current_length += section_length
        
        # Xử lý phần còn lại trong current_chunk
        if current_chunk:
            chunks.append('\n'.join(current_chunk))
        
        # Đảm bảo các chunk có overlap
        final_chunks = []
        for i in range(len(chunks)):
            if i > 0:
                # Thêm phần overlap từ chunk trước
                prev_chunk = chunks[i-1]
                overlap_text = prev_chunk[-chunk_overlap:] if len(prev_chunk) > chunk_overlap else prev_chunk
                chunks[i] = overlap_text + "\n" + chunks[i]
            final_chunks.append(chunks[i].strip())
            
        return final_chunks
    
    if parts and not re.match(dieu_le_pattern.strip(), parts[0].strip()):
        if len(parts[0].strip()) > 0:
            sub_chunks = split_content_respecting_numbers(parts[0].strip(), chunk_size, chunk_overlap)
            for sc in sub_chunks:
                chunks.append({
                    "text": sc,
                    "metadata": {
                        "dieu_le": "Giới thiệu/Quy định chung",
                        "source": source_name,
                        "file_path": source_file
                    }
                })
        
        # Bắt đầu xử lý từ phần tử tiếp theo
        start_index = 1 
    else:
        start_index = 0

    i = start_index
    while i < len(parts):
        if re.match(r"(Điều\s+\d+)", parts[i].strip()):
            # Đây là một tiêu đề Điều lệ
            # Nếu có nội dung tích lũy từ Điều lệ trước đó, xử lý nó
            if current_content.strip():
                sub_chunks = split_content_respecting_numbers(current_content.strip(), chunk_size, chunk_overlap)
                
                for j, sc in enumerate(sub_chunks):
                    # Đảm bảo mỗi chunk con đều có tiêu đề Điều lệ
                    prefix = f"{current_dieu_le_title}"
                    if j > 0:
                        prefix += " (tiếp theo)"
                    
                    final_chunk_text = f"{prefix}\n{sc}"
                    chunks.append({
                        "text": final_chunk_text,
                        "metadata": {
                            "dieu_le": current_dieu_le_title.split(':')[0].strip(),
                            "source": source_name,
                            "file_path": source_file
                        }
                    })
                current_content = "" # Reset nội dung sau khi đã xử lý
            
            current_dieu_le_title = parts[i].strip()
            # Tìm nội dung của Điều lệ này
            if i + 1 < len(parts) and not re.match(dieu_le_pattern.strip(), parts[i+1].strip()):
                 current_content = parts[i+1].strip()
                 i += 2 # Di chuyển qua tiêu đề và nội dung của nó
            else:
                 i += 1 # Chỉ di chuyển qua tiêu đề
                 
        else: # Đây là phần nội dung không phải tiêu đề Điều lệ
            current_content += parts[i]
            i += 1
            
    # Xử lý nội dung của Điều lệ cuối cùng (nếu có)
    if current_content.strip():
        sub_chunks = split_content_respecting_numbers(current_content.strip(), chunk_size, chunk_overlap)
        for j, sc in enumerate(sub_chunks):
            prefix = f"{current_dieu_le_title}"
            if j > 0:
                prefix += " (tiếp theo)"
            final_chunk_text = f"{prefix}\n{sc}"
            chunks.append({
                "text": final_chunk_text,
                "metadata": {
                    "dieu_le": current_dieu_le_title.split(':')[0].strip(),
                    "source": source_name,
                    "file_path": source_file
                }
            })

    return chunks

In [25]:
print("Đang đọc các file văn bản từ thư mục processed...")
documents = []

# Đọc tất cả các file .txt trong thư mục processed
for filename in os.listdir(PROCESSED_DATA_DIR):
    if filename.endswith('.txt'):
        file_path = os.path.join(PROCESSED_DATA_DIR, filename)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text_content = f.read()
            # Tạo chunks cho từng file với thông tin nguồn
            file_chunks = chunk_document_by_dieu_le(
                text_content, 
                CHUNK_SIZE, 
                CHUNK_OVERLAP, 
                source_file=file_path
            )
            documents.extend(file_chunks)
            print(f"Đã xử lý file: {filename} - Tạo được {len(file_chunks)} chunks")
        except Exception as e:
            print(f"Lỗi khi xử lý file {filename}: {e}")

print(f"\nTổng số chunks từ tất cả các file: {len(documents)}")
print("\nMột số chunks mẫu:")
for i, doc in enumerate(documents[:6]):
    print(f"\n--- Chunk {i+1} ---")
    print(f"Metadata: {doc['metadata']}")
    print(f"Nội dung (dài {len(doc['text'])} ký tự):\n{doc['text'][:1000]}...")

Đang đọc các file văn bản từ thư mục processed...
Đã xử lý file: HD hoc chuyen tiep ky su 180 TC_Final.txt - Tạo được 8 chunks
Đã xử lý file: QD ban hanh QD chuyen doi hoc phan tuong duong.txt - Tạo được 14 chunks
Đã xử lý file: QD ban hanh QD to chuc day hoc tren nen tang CN ket noi - truc tuyen.txt - Tạo được 25 chunks
Đã xử lý file: QD ban hanh QD to chuc thi Truc tuyen.txt - Tạo được 27 chunks
Đã xử lý file: QD_ngoai_ngu_tu_K65_CQ_final.txt - Tạo được 43 chunks
Đã xử lý file: QD_ngoai_ngu_tu_K68_CQ_final.txt - Tạo được 41 chunks
Đã xử lý file: Quy chế CTSV ĐHBK Hà Nội 2025.3.10_final.txt - Tạo được 67 chunks
Đã xử lý file: Quy_che_25.txt - Tạo được 103 chunks
Đã xử lý file: QĐ Học bổng KKHT 2023.txt - Tạo được 9 chunks
Đã xử lý file: QĐ Học bổng Trần Đại Nghĩa 2023.txt - Tạo được 16 chunks

Tổng số chunks từ tất cả các file: 353

Một số chunks mẫu:

--- Chunk 1 ---
Metadata: {'dieu_le': 'Giới thiệu/Quy định chung', 'source': 'HD hoc chuyen tiep ky su 180 TC_Final', 'file_path': 'C:

## 2. Tải và Xử Lý Dữ Liệu

Bây giờ chúng ta sẽ sử dụng hàm chunking đã định nghĩa để xử lý tất cả các file văn bản trong thư mục `processed`. Mỗi chunk sẽ được gắn thêm metadata để lưu trữ:

- **source**: Tên file gốc (không có phần mở rộng)
- **file_path**: Đường dẫn đầy đủ đến file gốc
- **dieu_le**: Số điều lệ hoặc phần của văn bản

Điều này giúp chúng ta có thể:
1. Truy xuất nguồn gốc của mỗi đoạn văn bản
2. Phân biệt được chunk từ file nào
3. Lọc hoặc tìm kiếm theo từng nguồn cụ thể

In [26]:
def chunk_document_by_dieu_le(text: str, chunk_size: int, chunk_overlap: int, source_file: str = None) -> List[Dict]:
    chunks = []
    dieu_le_pattern = r"(Điều\s+\d+)(?::\s*.*?)?\n"
    parts = re.split(dieu_le_pattern, text)
    
    # Xử lý tên nguồn từ tên file
    if source_file:
        source_name = os.path.splitext(os.path.basename(source_file))[0]
    else:
        source_name = "unknown_source"
    
    current_dieu_le_title = "Giới thiệu/Quy định chung" # Cho phần đầu tài liệu trước Điều lệ đầu tiên
    current_content = ""
    
    def split_content_respecting_numbers(content: str, chunk_size: int, chunk_overlap: int) -> List[str]:
        # Pattern để nhận diện các loại điểm đánh số
        number_patterns = [
            r'(?m)^\s*\d+\.',  # Số và dấu chấm (1., 2., ...)
            r'(?m)^\s*[a-z]\)',  # Chữ cái thường và dấu đóng ngoặc (a), b), ...)
            r'(?m)^\s*[ivx]+\)',  # Số La Mã và dấu đóng ngoặc (i), ii), ...)
        ]
        
        # Tìm tất cả các vị trí bắt đầu của các điểm được đánh số và chia đoạn
        sections = []
        last_pos = 0
        content_lines = content.split('\n')
        current_section = []
        
        for line in content_lines:
            # Kiểm tra xem dòng có bắt đầu bằng số hoặc điểm đánh số không
            is_numbered = any(re.match(pattern, line) for pattern in number_patterns)
            
            if is_numbered and current_section:
                # Lưu section hiện tại và bắt đầu section mới
                sections.append('\n'.join(current_section))
                current_section = [line]
            else:
                current_section.append(line)
        
        # Thêm section cuối cùng
        if current_section:
            sections.append('\n'.join(current_section))
        
        # Gộp các section thành chunks có kích thước phù hợp
        chunks = []
        current_chunk = []
        current_length = 0
        
        for section in sections:
            section_length = len(section)
            
            # Nếu section dài hơn chunk_size, chia nhỏ section này
            if section_length > chunk_size:
                # Xử lý nội dung còn lại trong current_chunk trước
                if current_chunk:
                    chunks.append('\n'.join(current_chunk))
                    current_chunk = []
                    current_length = 0
                
                # Chia section dài thành các phần nhỏ hơn
                words = section.split()
                current_part = []
                current_part_length = 0
                
                for word in words:
                    word_length = len(word) + 1  # +1 cho khoảng trắng
                    if current_part_length + word_length > chunk_size and current_part:
                        chunks.append(' '.join(current_part))
                        # Giữ lại một phần overlap cho chunk tiếp theo
                        overlap_size = min(chunk_overlap, current_part_length)
                        current_part = current_part[-int(overlap_size/10):]  # Giữ lại khoảng 10% từ cuối
                        current_part_length = sum(len(w) + 1 for w in current_part)
                    current_part.append(word)
                    current_part_length += word_length
                
                if current_part:
                    chunks.append(' '.join(current_part))
            
            # Nếu thêm section hiện tại vào chunk hiện tại sẽ vượt quá kích thước
            elif current_length + section_length > chunk_size and current_chunk:
                chunks.append('\n'.join(current_chunk))
                current_chunk = [section]
                current_length = section_length
            else:
                current_chunk.append(section)
                current_length += section_length
        
        # Xử lý phần còn lại trong current_chunk
        if current_chunk:
            chunks.append('\n'.join(current_chunk))
        
        # Đảm bảo các chunk có overlap
        final_chunks = []
        for i in range(len(chunks)):
            if i > 0:
                # Thêm phần overlap từ chunk trước
                prev_chunk = chunks[i-1]
                overlap_text = prev_chunk[-chunk_overlap:] if len(prev_chunk) > chunk_overlap else prev_chunk
                chunks[i] = overlap_text + "\n" + chunks[i]
            final_chunks.append(chunks[i].strip())
            
        return final_chunks
    
    if parts and not re.match(dieu_le_pattern.strip(), parts[0].strip()):
        if len(parts[0].strip()) > 0:
            sub_chunks = split_content_respecting_numbers(parts[0].strip(), chunk_size, chunk_overlap)
            for sc in sub_chunks:
                chunks.append({
                    "text": sc,
                    "metadata": {
                        "dieu_le": "Giới thiệu/Quy định chung",
                        "source": source_name,
                        "file_path": source_file
                    }
                })
        
        # Bắt đầu xử lý từ phần tử tiếp theo
        start_index = 1 
    else:
        start_index = 0

    i = start_index
    while i < len(parts):
        if re.match(r"(Điều\s+\d+)", parts[i].strip()):
            # Đây là một tiêu đề Điều lệ
            # Nếu có nội dung tích lũy từ Điều lệ trước đó, xử lý nó
            if current_content.strip():
                sub_chunks = split_content_respecting_numbers(current_content.strip(), chunk_size, chunk_overlap)
                
                for j, sc in enumerate(sub_chunks):
                    # Đảm bảo mỗi chunk con đều có tiêu đề Điều lệ
                    prefix = f"{current_dieu_le_title}"
                    if j > 0:
                        prefix += " (tiếp theo)"
                    
                    final_chunk_text = f"{prefix}\n{sc}"
                    chunks.append({
                        "text": final_chunk_text,
                        "metadata": {
                            "dieu_le": current_dieu_le_title.split(':')[0].strip(),
                            "source": source_name,
                            "file_path": source_file
                        }
                    })
                current_content = "" # Reset nội dung sau khi đã xử lý
            
            current_dieu_le_title = parts[i].strip()
            # Tìm nội dung của Điều lệ này
            if i + 1 < len(parts) and not re.match(dieu_le_pattern.strip(), parts[i+1].strip()):
                 current_content = parts[i+1].strip()
                 i += 2 # Di chuyển qua tiêu đề và nội dung của nó
            else:
                 i += 1 # Chỉ di chuyển qua tiêu đề
                 
        else: # Đây là phần nội dung không phải tiêu đề Điều lệ
            current_content += parts[i]
            i += 1
            
    # Xử lý nội dung của Điều lệ cuối cùng (nếu có)
    if current_content.strip():
        sub_chunks = split_content_respecting_numbers(current_content.strip(), chunk_size, chunk_overlap)
        for j, sc in enumerate(sub_chunks):
            prefix = f"{current_dieu_le_title}"
            if j > 0:
                prefix += " (tiếp theo)"
            final_chunk_text = f"{prefix}\n{sc}"
            chunks.append({
                "text": final_chunk_text,
                "metadata": {
                    "dieu_le": current_dieu_le_title.split(':')[0].strip(),
                    "source": source_name,
                    "file_path": source_file
                }
            })

    return chunks

## 3. Tải Mô Hình Embedding

Chúng ta tải mô hình `SentenceTransformer` từ Hugging Face. Mô hình này sẽ nhận đầu vào là một đoạn văn bản và trả về một vector (một mảng các con số) có chiều dài lớn (ví dụ: 1024 chiều). Các đoạn văn bản có ý nghĩa tương tự nhau sẽ có vector gần nhau trong không gian vector.

In [27]:
print(f"\n4. Tải mô hình Embedding: {EMBEDDING_MODEL_NAME}")
try:
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
    print("Đã tải mô hình embedding thành công.")
except Exception as e:
    print(f"Lỗi khi tải mô hình embedding: {e}")
    print("Đảm bảo bạn có kết nối internet hoặc mô hình đã được tải xuống cục bộ.")
    exit()
    



4. Tải mô hình Embedding: intfloat/multilingual-e5-large
Đã tải mô hình embedding thành công.
Đã tải mô hình embedding thành công.


## 4. Khởi Tạo và Lưu Trữ vào Vector Database (ChromaDB)

Đây là bước chúng ta xây dựng "bộ não" cho hệ thống retrieval.

1.  **Khởi tạo Client:** Tạo một `PersistentClient` của ChromaDB, chỉ định đường dẫn để lưu dữ liệu xuống đĩa. Điều này đảm bảo dữ liệu không bị mất khi notebook tắt.
2.  **Tạo Collection:** Một `collection` trong ChromaDB giống như một "bảng" trong cơ sở dữ liệu truyền thống. Chúng ta tạo một collection mới, xóa collection cũ nếu có để đảm bảo dữ liệu luôn được làm mới.
3.  **Xử lý theo lô (Batch Processing):** Để tránh quá tải bộ nhớ, chúng ta không tạo embedding cho tất cả các chunk cùng một lúc. Thay vào đó, chúng ta lặp qua danh sách `documents` theo từng lô (ví dụ: 100 chunk một lần).
4.  **Tạo Embeddings & ID:** Với mỗi lô, chúng ta tạo embeddings và các ID duy nhất cho từng chunk.
5.  **Thêm vào Collection:** Dùng lệnh `collection.add()` để lưu đồng thời `embeddings`, `documents` (nội dung text), `metadatas`, và `ids` vào ChromaDB.


In [28]:
print(f"Khởi tạo ChromaDB tại: {CHROMA_DB_PATH}")

try:
    # Khởi tạo PersistentClient để lưu trữ dữ liệu trên đĩa
    client = chromadb.PersistentClient(path=CHROMA_DB_PATH)
    
    # Xóa collection cũ nếu tồn tại
    try:
        client.delete_collection(name="quy_che_sinh_vien_rag")
        print("Đã xóa collection cũ.")
    except Exception as e:
        print(f"Collection chưa tồn tại hoặc đã được xóa: {e}")
    
    # Tạo collection mới
    collection = client.create_collection(name="quy_che_sinh_vien_rag")
    print(f"Đã tạo collection mới: '{collection.name}'")
    
    print("\nBắt đầu tạo embeddings và thêm vào ChromaDB...")
    batch_size = 100  # Xử lý theo lô để tiết kiệm bộ nhớ và thời gian
    
    for i in range(0, len(documents), batch_size):
        batch_docs = documents[i:i + batch_size]
        
        # Trích xuất văn bản và metadata từ các chunk trong lô hiện tại
        texts_batch = [doc['text'] for doc in batch_docs]
        metadatas_batch = [doc['metadata'] for doc in batch_docs]
        
        # Tạo embeddings cho lô văn bản này
        embeddings_batch = embedding_model.encode(texts_batch, convert_to_tensor=False).tolist()
        
        # Tạo IDs duy nhất cho mỗi tài liệu (chunk) trong lô
        ids_batch = [str(uuid.uuid4()) for _ in texts_batch]
        
        # Thêm vào ChromaDB
        try:
            collection.add(
                embeddings=embeddings_batch,
                documents=texts_batch,
                metadatas=metadatas_batch,
                ids=ids_batch
            )
            print(f"Đã thêm thành công lô {i // batch_size + 1}/{len(documents) // batch_size + (1 if len(documents) % batch_size != 0 else 0)} ({len(texts_batch)} documents).")
        except Exception as e:
            print(f"Lỗi khi thêm lô {i // batch_size + 1} vào ChromaDB: {e}")
            raise  # Dừng lại nếu có lỗi khi thêm dữ liệu

    print("\nQuá trình tạo và lưu trữ Vector Database đã hoàn tất.")
    print(f"Tổng số document trong ChromaDB collection '{collection.name}': {collection.count()}")

except Exception as e:
    print(f"Lỗi khi làm việc với ChromaDB: {e}")
    print("Vui lòng kiểm tra lại cấu hình và thử lại.")

Khởi tạo ChromaDB tại: C:\Users\PC\Desktop\Project2\vectorstore\chroma_db
Đã xóa collection cũ.
Đã tạo collection mới: 'quy_che_sinh_vien_rag'

Bắt đầu tạo embeddings và thêm vào ChromaDB...
Đã tạo collection mới: 'quy_che_sinh_vien_rag'

Bắt đầu tạo embeddings và thêm vào ChromaDB...
Đã thêm thành công lô 1/4 (100 documents).
Đã thêm thành công lô 1/4 (100 documents).
Đã thêm thành công lô 2/4 (100 documents).
Đã thêm thành công lô 2/4 (100 documents).
Đã thêm thành công lô 3/4 (100 documents).
Đã thêm thành công lô 3/4 (100 documents).
Đã thêm thành công lô 4/4 (53 documents).

Quá trình tạo và lưu trữ Vector Database đã hoàn tất.
Tổng số document trong ChromaDB collection 'quy_che_sinh_vien_rag': 353
Đã thêm thành công lô 4/4 (53 documents).

Quá trình tạo và lưu trữ Vector Database đã hoàn tất.
Tổng số document trong ChromaDB collection 'quy_che_sinh_vien_rag': 353


## 5. Kiểm Tra Vector Database (Tùy chọn)

Sau khi đã lập chỉ mục, đây là một bước sanity-check quan trọng để đảm bảo mọi thứ hoạt động đúng.

1.  **Tạo câu hỏi mẫu:** Đặt một câu hỏi (`test_query`) mà bạn mong muốn tìm được câu trả lời trong tài liệu.
2.  **Tạo embedding cho câu hỏi:** Dùng chính mô hình embedding đã sử dụng để chuyển câu hỏi thành vector.
3.  **Truy vấn (Query):** Dùng `collection.query()` để tìm kiếm trong ChromaDB. ChromaDB sẽ tính toán sự tương đồng (khoảng cách) giữa vector câu hỏi và tất cả các vector của chunk trong collection, sau đó trả về `n_results` chunk có khoảng cách nhỏ nhất (tức là liên quan nhất).
4.  **Phân tích kết quả:** In ra nội dung, metadata và khoảng cách của các chunk được tìm thấy để đánh giá chất lượng retrieval. **Khoảng cách (Distance) càng nhỏ, độ liên quan càng cao.**


In [29]:
try:
    test_query = "Học sinh được bảo lưu kết quả học tập trong những trường hợp nào?"
    query_embedding = embedding_model.encode([test_query]).tolist()
    
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=15,
        include=['documents', 'distances', 'metadatas']
    )
    
    print(f"\nTruy vấn mẫu: '{test_query}'")
    for i, (doc, dist, meta) in enumerate(zip(results['documents'][0], results['distances'][0], results['metadatas'][0])):
        print(f"\n--- Kết quả {i+1} ---")
        print(f"Khoảng cách (Distance): {dist}")
        print(f"Metadata: {meta}")
        print(f"Nội dung: {doc[:500]}...") # Chỉ hiển thị 500 ký tự đầu
except Exception as e:
    print(f"Lỗi khi thực hiện truy vấn kiểm tra: {e}")
    print("Vui lòng kiểm tra lại cấu hình và dữ liệu trong ChromaDB.")

print("\nNotebook 02_vector_db_indexing.ipynb đã hoàn thành!")


Truy vấn mẫu: 'Học sinh được bảo lưu kết quả học tập trong những trường hợp nào?'

--- Kết quả 1 ---
Khoảng cách (Distance): 0.34488123655319214
Metadata: {'dieu_le': 'Giới thiệu/Quy định chung', 'file_path': 'C:\\Users\\PC\\Desktop\\Project2\\data\\processed\\Quy chế CTSV ĐHBK Hà Nội 2025.3.10_final.txt', 'source': 'Quy chế CTSV ĐHBK Hà Nội 2025.3.10_final'}
Nội dung: ăn bằng tốt nghiệp kèm phụ lục văn bằng;
c) Kết quả rèn luyện trong quá trình học tập của sinh viên.
4. Hồ sơ nhập học và hồ sơ quản lý quá trình học tập, rèn luyện của sinh viên
được lập chậm nhất 15 ngày sau khi sinh viên làm thủ tục nhập học. Hồ sơ tốt nghiệp
của sinh viên được hoàn thiện chậm nhất 7 ngày trước thời điểm sinh viên được nhận bằng
tốt nghiệp.
5. Hồ sơ văn bản giấy của sinh viên được quản lý tập trung; hồ sơ dạng tài liệu
điện tử được lưu trữ và cập nhật đầy đủ, thường xuyên trê...

--- Kết quả 2 ---
Khoảng cách (Distance): 0.3515351414680481
Metadata: {'source': 'Quy_che_25', 'file_path': 'C:\\Users\\P


## 6. Cải Thiện Kết Quả với Re-ranking (tùy chọn)

**Vấn đề:** Tìm kiếm vector (retrieval) rất nhanh và hiệu quả trong việc tìm ra các đoạn văn bản *có liên quan về mặt ngữ nghĩa*. Tuy nhiên, nó có thể không phải lúc nào cũng là đoạn văn bản *chính xác nhất* để trả lời câu hỏi.

**Giải pháp:** **Re-ranking**.
1.  **Lấy top-k kết quả:** Chúng ta lấy một số lượng lớn các kết quả từ ChromaDB (ví dụ: 15 kết quả).
2.  **Sử dụng mô hình chuyên biệt:** Chúng ta dùng một mô hình Cross-Encoder (như `bert-multilingual-passage-reranking-msmarco`). Mô hình này nhận đầu vào là một cặp `(câu hỏi, đoạn văn bản)` và đưa ra một điểm số duy nhất thể hiện mức độ liên quan trực tiếp.
3.  **Sắp xếp lại:** Dựa trên điểm số này, chúng ta sắp xếp lại 15 kết quả ban đầu và chỉ lấy top 3 hoặc top 5 kết quả có điểm cao nhất.

Đây là một kỹ thuật nâng cao giúp tăng đáng kể độ chính xác của ngữ cảnh được cung cấp cho LLM ở bước cuối cùng.


In [30]:


# Đường dẫn lưu mô hình re-ranking
reranking_model_path = os.path.join(MODELS_PATH, 'reranking-msmarco')
# Tải hoặc load mô hình và tokenizer
print("Đang tải/load mô hình re-ranking...")
try:
    # Thử load mô hình từ local trước
    rerank_tokenizer = AutoTokenizer.from_pretrained(reranking_model_path)
    rerank_model = AutoModelForSequenceClassification.from_pretrained(reranking_model_path)
    print("Đã load mô hình re-ranking từ local thành công.")
except Exception as e:
    print("Chưa có mô hình local, đang tải mô hình từ Hugging Face...")
    # Tải mô hình từ Hugging Face và lưu xuống local
    rerank_tokenizer = AutoTokenizer.from_pretrained(RERANKING_MODEL_NAME)
    rerank_model = AutoModelForSequenceClassification.from_pretrained(RERANKING_MODEL_NAME)
    
    # Lưu mô hình và tokenizer
    print("Đang lưu mô hình re-ranking vào local...")
    rerank_tokenizer.save_pretrained(reranking_model_path)
    rerank_model.save_pretrained(reranking_model_path)
    print(f"Đã lưu mô hình re-ranking tại: {reranking_model_path}")

def rerank_passages(query: str, passages: list, n_results: int = 3) -> list:
    features = []
    for passage in passages:
        inputs = rerank_tokenizer.encode_plus(
            query,
            passage,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        features.append(inputs)
    
    scores = []
    rerank_model.eval()
    with torch.no_grad():
        for feat in features:
            outputs = rerank_model(**feat)
            # Dùng softmax để lấy xác suất “liên quan”
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
            score = probs[0, -1].item()  # class cuối cùng
            scores.append(score)
    
    # Sắp xếp lại
    ranked_passages = sorted(zip(passages, scores), key=lambda x: x[1], reverse=True)
    return ranked_passages[:n_results]

# Áp dụng re-ranking cho kết quả từ ChromaDB
retrieved_docs = results['documents'][0]  # Lấy 15 kết quả từ ChromaDB
reranked_results = rerank_passages(test_query, retrieved_docs)

print(f"\nKết quả sau khi re-ranking (top 3 đoạn liên quan nhất):")
for i, (doc, score) in enumerate(reranked_results):
    print(f"\n--- Kết quả Re-ranked {i+1} ---")
    print(f"Điểm liên quan (Relevance Score): {score:.4f}")
    print(f"Nội dung: {doc[:1000]}...")  # Hiển thị 1000 ký tự đầu

Đang tải/load mô hình re-ranking...
Đã load mô hình re-ranking từ local thành công.

Kết quả sau khi re-ranking (top 3 đoạn liên quan nhất):

--- Kết quả Re-ranked 1 ---
Điểm liên quan (Relevance Score): 0.9875
Nội dung: thời hạn 5 năm kể từ ngày khóa học kết thúc. 
Điều 35. Nghỉ học tạm thời và bảo lưu kết quả học tập 
1. Đối với học viên được điều động vào lực lượng vũ trang, đi làm nghĩa vụ quốc gia, 
quốc tế, thời gian nghỉ học tạm thời (là thời gian được ghi trong quyết định của cấp có 
thẩm quyền) không tính vào thời gian học tại ĐHBK Hà Nội.  
2. Học viên bị ốm hoặc tai nạn buộc phải điều trị trong thời gian dài, hoặc nghỉ thai 
sản theo quy định được phép nghỉ học tạm thời. T hời gian nghỉ học tạm thời (là thời gian 
được ghi trong giấy xác nhận của cơ sở khám, chữa bệnh có thẩm quyền theo quy định của 
Bộ Y tế) không tính vào thời gian chậm tiến độ nếu tổng cộng không vượt quá 2 học kỳ 
chính; thời gian nghỉ trên 2 học kỳ được tính vào thời gian học chậm tiến độ....

--- Kết q